Step 1. Data Acquisition

[https://www.kaggle.com/datasets/kyanyoga/sample-sales-data](https://)

In [2]:
import pandas as pd

In [3]:

# Read the CSV file using ISO-8859-1 encoding
df = pd.read_csv("data/sales_data_sample.csv", encoding='ISO-8859-1')

# Display the first few rows of the DataFrame
print(df.head())


   ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES  \
0        10107               30      95.70                2  2871.00   
1        10121               34      81.35                5  2765.90   
2        10134               41      94.74                2  3884.34   
3        10145               45      83.26                6  3746.70   
4        10159               49     100.00               14  5205.27   

         ORDERDATE   STATUS  QTR_ID  MONTH_ID  YEAR_ID  ...  \
0   2/24/2003 0:00  Shipped       1         2     2003  ...   
1    5/7/2003 0:00  Shipped       2         5     2003  ...   
2    7/1/2003 0:00  Shipped       3         7     2003  ...   
3   8/25/2003 0:00  Shipped       3         8     2003  ...   
4  10/10/2003 0:00  Shipped       4        10     2003  ...   

                    ADDRESSLINE1  ADDRESSLINE2           CITY STATE  \
0        897 Long Airport Avenue           NaN            NYC    NY   
1             59 rue de l'Abbaye           NaN

setting up database

In [4]:
from sqlalchemy import create_engine
from sqlalchemy import text

temp_db = create_engine('sqlite:///:memory:', echo=True)

data = df.to_sql(name='Sales', con=temp_db)

2024-08-25 22:10:22,007 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-25 22:10:22,038 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-08-25 22:10:22,040 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-25 22:10:22,045 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2024-08-25 22:10:22,048 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-25 22:10:22,062 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

Creating the connection to SQL and querying the database

In [5]:
with temp_db.connect() as conn:
  result = conn.execute(text("Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1"))

result.all()

2024-08-25 22:10:30,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-25 22:10:30,363 INFO sqlalchemy.engine.Engine Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1
2024-08-25 22:10:30,365 INFO sqlalchemy.engine.Engine [generated in 0.00532s] ()
2024-08-25 22:10:30,370 INFO sqlalchemy.engine.Engine ROLLBACK


[(10407, 14082.8)]

Setting up Open AI account and informing GPT about our data

In [7]:
def create_table_definition_prompt(df):
    """
    This function returns a prompt that informs GPT that we want to work with SQL Tables
    """
    prompt = '''### sqlite SQL table, with its properties:

    # Sales({})
    #
    #
    '''.format(",".join(str(x) for x in df.columns))

    return prompt

print(create_table_definition_prompt(df))


### sqlite SQL table, with its properties:

    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    #
    


 Taking natural language requests from the end user and combining prompts

In [8]:
def prompt_input():
  nlp_text = input("Enter information you want to obtain: ")
  return nlp_text

nlp_text = prompt_input()

In [9]:
def combine_prompts(df, query_prompt):
  definition = create_table_definition_prompt(df)
  query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
  return definition+query_init_string

combine_prompts(df, nlp_text)

'### sqlite SQL table, with its properties:\n\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    #\n    ### A query to answer: Get me detail against sales\nSELECT'

In [12]:
from openai import OpenAI

client = OpenAI(
    base_url="http://172.31.0.1:11434/v1",

    # required but ignored
    api_key='ollama',
)

In [15]:
response = client.chat.completions.create(
    model="llama3:8b-instruct-fp16",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": combine_prompts(df, nlp_text)}
    ],
    temperature=0,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop=["#", ";"],
    stream=False,
)

print(response)


ChatCompletion(id='chatcmpl-956', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="You want a SQL query to get details about sales. Here's a possible query:\n\n```\nSELECT \n  ORDERNUMBER,\n  QUANTITYORDERED,\n  PRICEEACH,\n  ORDERLINENUMBER,\n  SALES,\n  ORDERDATE,\n  STATUS,\n  QTR_ID,\n  MONTH_ID,\n  YEAR_ID,\n  PRODUCTLINE,\n  MSRP,\n  PRODUCTCODE,\n  CUSTOMERNAME,\n  PHONE,\n  ADDRESSLINE1,\n  ADDRESSLINE2,\n  CITY,\n  STATE,\n  POSTALCODE,\n  COUNTRY,\n  TERRITORY,\n  CONTACTLASTNAME,\n  CONTACTFIRSTNAME,\n  DEALSIZE\nFROM \n  Sales", role='assistant', function_call=None, tool_calls=None))], created=1724606463, model='llama3:8b-instruct-fp16', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=136, prompt_tokens=135, total_tokens=271))


building a function to parse the response section and pass the same into our database

In [23]:
import re

def handle_response(response):
    content = response.choices[0].message.content
    # Extract the SQL query using regex
    match = re.search(r'```\s*\n(SELECT[\s\S]*?)```', content, re.IGNORECASE)
    if match:
        query = match.group(1).strip()
    else:
        # If no SQL found, use the entire content (not recommended, but as a fallback)
        query = content

    # Ensure the query starts with SELECT
    if not query.upper().startswith("SELECT"):
        query = "SELECT" + query

    return query

query = handle_response(response)

with temp_db.connect() as conn:
    result = conn.execute(text(query))

2024-08-25 22:25:36,932 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-25 22:25:36,936 INFO sqlalchemy.engine.Engine SELECTYou want a SQL query to get details about sales. Here's a possible query:

```
SELECT 
  ORDERNUMBER,
  QUANTITYORDERED,
  PRICEEACH,
  ORDERLINENUMBER,
  SALES,
  ORDERDATE,
  STATUS,
  QTR_ID,
  MONTH_ID,
  YEAR_ID,
  PRODUCTLINE,
  MSRP,
  PRODUCTCODE,
  CUSTOMERNAME,
  PHONE,
  ADDRESSLINE1,
  ADDRESSLINE2,
  CITY,
  STATE,
  POSTALCODE,
  COUNTRY,
  TERRITORY,
  CONTACTLASTNAME,
  CONTACTFIRSTNAME,
  DEALSIZE
FROM 
  Sales
2024-08-25 22:25:36,938 INFO sqlalchemy.engine.Engine [generated in 0.00667s] ()
2024-08-25 22:25:36,941 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) near "SELECTYou": syntax error
[SQL: SELECTYou want a SQL query to get details about sales. Here's a possible query:

```
SELECT 
  ORDERNUMBER,
  QUANTITYORDERED,
  PRICEEACH,
  ORDERLINENUMBER,
  SALES,
  ORDERDATE,
  STATUS,
  QTR_ID,
  MONTH_ID,
  YEAR_ID,
  PRODUCTLINE,
  MSRP,
  PRODUCTCODE,
  CUSTOMERNAME,
  PHONE,
  ADDRESSLINE1,
  ADDRESSLINE2,
  CITY,
  STATE,
  POSTALCODE,
  COUNTRY,
  TERRITORY,
  CONTACTLASTNAME,
  CONTACTFIRSTNAME,
  DEALSIZE
FROM 
  Sales]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [24]:
query

"SELECTYou want a SQL query to get details about sales. Here's a possible query:\n\n```\nSELECT \n  ORDERNUMBER,\n  QUANTITYORDERED,\n  PRICEEACH,\n  ORDERLINENUMBER,\n  SALES,\n  ORDERDATE,\n  STATUS,\n  QTR_ID,\n  MONTH_ID,\n  YEAR_ID,\n  PRODUCTLINE,\n  MSRP,\n  PRODUCTCODE,\n  CUSTOMERNAME,\n  PHONE,\n  ADDRESSLINE1,\n  ADDRESSLINE2,\n  CITY,\n  STATE,\n  POSTALCODE,\n  COUNTRY,\n  TERRITORY,\n  CONTACTLASTNAME,\n  CONTACTFIRSTNAME,\n  DEALSIZE\nFROM \n  Sales"